In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Input, Flatten, Reshape, BatchNormalization, MaxPooling2D, LeakyReLU, PReLU
from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay


import numpy as np
from glob import glob
from tqdm import tqdm


import os
import matplotlib.pyplot as plt
from cv2 import COLOR_BGR2LAB, COLOR_BGR2GRAY
import cv2
from PIL import Image
from random import shuffle



In [2]:
import shutil

In [3]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
# !unzip -q /content/drive/MyDrive/Coding/Data/caltech_256.zip

In [5]:
!tar -xvf /content/drive/MyDrive/Coding/Data/ILSVRC2013_DET_train.tar

ILSVRC2013_DET_train/
ILSVRC2013_DET_train/n02109047.tar
ILSVRC2013_DET_train/n02112018.tar
ILSVRC2013_DET_train/n04591157.tar
ILSVRC2013_DET_train/n02105412.tar
ILSVRC2013_DET_train/n02077923.tar
ILSVRC2013_DET_train/n03494278.tar
ILSVRC2013_DET_train/n01742172.tar
ILSVRC2013_DET_train/n02086646.tar
ILSVRC2013_DET_train/n03785016.tar
ILSVRC2013_DET_train/n03841666.tar
ILSVRC2013_DET_train/n02113799.tar
ILSVRC2013_DET_train/n02981792.tar
ILSVRC2013_DET_train/n03481172.tar
ILSVRC2013_DET_train/n04332243.tar
ILSVRC2013_DET_train/n04039381.tar
ILSVRC2013_DET_train/n02084071.tar
ILSVRC2013_DET_train/n01806143.tar
ILSVRC2013_DET_train/n01614925.tar
ILSVRC2013_DET_train/n03777568.tar
ILSVRC2013_DET_train/n07768694.tar
ILSVRC2013_DET_train/n03599486.tar
ILSVRC2013_DET_train/n01622779.tar
ILSVRC2013_DET_train/n02093256.tar
ILSVRC2013_DET_train/ILSVRC2013_DET_train_extra10.tar
ILSVRC2013_DET_train/n07718472.tar
ILSVRC2013_DET_train/ILSVRC2013_DET_train_extra8.tar
ILSVRC2013_DET_train/n03594945.

In [6]:
# !unzip  /content/drive/MyDrive/Coding/Data/mirflickr25k.zip

In [7]:
def bw_image_remove(sample_path):
  counter = 0
  for i in sample_path:
    counter += 1
    sample_image = cv2.imread(i)
    Chan_A = sample_image[:,:,0]
    Chan_B = sample_image[:,:,1]
    Chan_C = sample_image[:,:,2]
    if (Chan_A == Chan_B).all():
      os.remove(i)

In [8]:
def give_me_sample(sample_path):
  rand_index = np.random.choice(len(sample_path))

  sample_img = cv2.imread(sample_path[rand_index])
  sample_img_2 = cv2.cvtColor(sample_img, cv2.COLOR_BGR2LAB)
  sample_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)

  sample_img = sample_img/255.0
  print(sample_img.max())
  print(sample_img.min())

  plt.imshow(sample_img)

  print(sample_img.shape)

In [9]:
batch_size = 8
epochs = 200
show_every = 10
image_size = (96,96)
H = image_size[0]
W = image_size[1]
# img_path = sample_path


In [10]:
def build_block(filter, kernel, strides, x):
    x = Conv2D(filters= filter, kernel_size= kernel, strides= strides, padding = "same")(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    return x


In [11]:
def build_gan_model():
    i = Input(shape = (H,W,1))
    x = Conv2D(64, kernel_size= 2, strides= 1, padding="same")(i)
    x = build_block(64,2,2,x=x)
    #112
    x = build_block(128,2,2,x=x)
    x_2 = x
    #56
    x = build_block(256,2,1,x=x)
    # x_1 = x
    # x = build_block(256,2,1,x=x)
    #28
    x = build_block(512,2,1,x=x)
    x = build_block(512,2,1,x=x)
    x = build_block(512,2,1,x=x)
    x = build_block(512,2,1,x=x)
    x = build_block(512,2,1,x=x)
    #28
    x = tf.nn.depth_to_space(x,2)
    
    #56
    x = build_block(256,2,1,x=x)
    x = build_block(256,2,2,x=x)
    # x = tf.keras.layers.add([x_1,x])
    x = tf.nn.depth_to_space(x,2)
    #112
    x = build_block(128,2,1,x=x)
    x = build_block(128,2,1,x=x)
    # x = tf.keras.layers.add([x_2,x])
    x = tf.nn.depth_to_space(x,2)
    
    
    x = build_block(64,3,1,x=x)
    x = Conv2D(2,1,1, activation= "tanh")(x)

    model = Model(inputs = i, outputs = x)

    return model

In [12]:
def build_dis_model():
  i = Input(shape = (H,W,3))
  x = Conv2D(filters=64, kernel_size=3, strides = 1, activation= LeakyReLU())(i)
  x = build_block(64,3,1,x)
  x = build_block(128,3,1,x)
  x = build_block(128,3,2,x)
  x = build_block(256,3,1,x)
  x = build_block(256,3,2,x)
  x = build_block(512,3,1,x)
  x = build_block(512,3,2,x)

  x = Flatten()(x)
  x = Dense(1024)(x)
  x = LeakyReLU()(x)
  x = Dense(1, activation = "sigmoid")(x)

  model = Model(inputs = i, outputs = x)
  return model

In [13]:
gan_model = build_gan_model()
dis_model = build_dis_model()

In [14]:
resnet_layer = ResNet50(include_top= False, weights= "imagenet", input_shape= (H,W,3))
resnet_model = Model(inputs = resnet_layer.input, outputs = resnet_layer.layers[32].output)
resnet_model.trainable = False

94781440/94765736 [==============================] - 1s 0us/step


In [15]:
# resnet_model.summary()

In [16]:
# gan_model.summary()

In [17]:
def lab_np(batch_size, batch, img_path):
    g_array = np.zeros((1, H,W,1))
    c_array = np.zeros((1,H,W,3))
    image_size = (96,96)
    for path in img_path[batch:batch + batch_size]:
        open_img = cv2.imread(path)
        open_img = cv2.resize(open_img, image_size)

        color_img = cv2.cvtColor(open_img, COLOR_BGR2LAB)
        color_img = color_img/255.0
        gray_img = color_img[:,:,0]

        gray_img = np.reshape(gray_img, (1,H,W,1))
        color_img = np.expand_dims(color_img, axis = 0)
        

        c_array = np.append(c_array, color_img, axis = 0)
        g_array = np.append(g_array, gray_img, axis = 0)

    c_array = np.delete(c_array, 0, axis = 0)
    g_array = np.delete(g_array, 0, axis = 0)
        
        
    return g_array, c_array

In [18]:
file_path = "/content/drive/MyDrive/Coding/image_result/gray_to_color/"
if not os.path.exists(file_path):
    os.makedirs(file_path)

In [19]:
check_path = "/content/drive/MyDrive/Coding/Check_point/gray_to_color"
if not os.path.exists(check_path):
    os.makedirs(check_path)

check_point = tf.train.Checkpoint(gan_model = gan_model, dis_model = dis_model)
check_point_manager = tf.train.CheckpointManager(checkpoint= check_point, directory=check_path, max_to_keep=5)

In [20]:
check_point.restore(check_point_manager.latest_checkpoint)

In [21]:
def plot_result(epoch, img_path, file_number):
    epoch = epoch
    
    rows = 4
    cols = 3

    fig, ax = plt.subplots(rows,cols, figsize = (12,12))
    fig.tight_layout()

    for i in range(rows):
        img_open = cv2.imread(img_path[i])
        img_open = cv2.resize(img_open,(H,W))
        real_img = cv2.cvtColor(img_open, cv2.COLOR_BGR2LAB)
        real_img = real_img/255.0

        
        gray_img = real_img[:,:,0]
        gray_img_for_imshow = real_img[:,:,0]

        gray_img = np.reshape(gray_img, (1,H,W,1))

        

        
        fake_img = gan_model(gray_img, training = False)
        fake_img = (fake_img + 1)/2.0
        
        fake_img = np.reshape(fake_img, (H,W,2))

        gray_img = np.reshape(gray_img, (H,W,1))
        fake_img = np.concatenate((gray_img, fake_img), axis = -1, dtype = "float32")

        fake_img = (fake_img*255).astype('uint8')
        fake_img = cv2.cvtColor(fake_img, cv2.COLOR_LAB2RGB)   
        
        real_img = (real_img*255).astype('uint8')
        real_img = cv2.cvtColor(real_img, cv2.COLOR_LAB2RGB)
        


        ax[i][0].imshow(gray_img_for_imshow, cmap = "gray")
        ax[i][1].imshow(fake_img)
        ax[i][2].imshow(real_img)

    while True:
      file_name = file_path + "file_sample_" + str(file_number) + f"/%d.png" %epoch
      
      if not os.path.exists(file_name):
        break     
      epoch += 2

    plt.savefig(file_name)
    plt.close()

    

In [22]:
discriminator_losses = tf.keras.losses.BinaryCrossentropy()
generator_losses = tf.keras.losses.BinaryCrossentropy()
content_losses = tf.keras.losses.MeanSquaredError()

learning_rate = PiecewiseConstantDecay(boundaries = [1000], values = [1e-3, 1e-4])

gen_optimizer = Adam(learning_rate = learning_rate)
dis_optimizer = Adam(learning_rate = learning_rate)


def content_loss(real_imgs, fake_imgs):

    real_imgs = tf.cast(real_imgs, dtype = tf.float32)
    fake_imgs = tf.cast(fake_imgs, dtype = tf.float32)

    
    real_imgs = preprocess_input(real_imgs)
    fake_imgs = preprocess_input(fake_imgs)

    how_real_classified = resnet_model(real_imgs)
    how_fake_classified = resnet_model(fake_imgs)

    Resnet_loss = content_losses(how_real_classified, how_fake_classified)

    actual_loss = content_losses(real_imgs, fake_imgs)

    return Resnet_loss, actual_loss
  


def discriminator_loss(real_output, fake_output):
    real_loss = discriminator_losses(tf.ones_like(real_output), real_output)
    fake_loss = discriminator_losses(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    
    fake_loss = generator_losses(tf.ones_like(fake_output), fake_output)
    
    return fake_loss

In [23]:
def train_step(real_imgs, gray_imgs):
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as dis_tape:
        fake_imgs = gan_model(gray_imgs, training = True)
        fake_imgs = (fake_imgs+1)/2.0
        fake_imgs = tf.concat((gray_imgs, fake_imgs), axis = -1)
        # fake_imgs = np.concatenate((gray_imgs, fake_imgs), axis = -1)

        c_loss, actual_loss = content_loss(real_imgs, fake_imgs)


        real_output = dis_model(real_imgs, training = True)
        fake_output = dis_model(fake_imgs, training = True)

        d_loss = discriminator_loss(real_output= real_output, fake_output= fake_output)
        g_loss = generator_loss(fake_output= fake_output)

        cg_loss = 10*actual_loss + 0.01*c_loss + 0.001*g_loss
        # cg_loss = c_loss + 0.001*g_loss
    

        g_gradient = gen_tape.gradient(cg_loss, gan_model.trainable_variables)
        d_gradient = dis_tape.gradient(d_loss, dis_model.trainable_variables)

        gen_optimizer.apply_gradients(zip(g_gradient, gan_model.trainable_variables))
        dis_optimizer.apply_gradients(zip(d_gradient, dis_model.trainable_variables))

    return cg_loss, d_loss

In [ ]:
import tarfile
d_losses = tf.keras.metrics.Mean(name="d_loss")
g_losses = tf.keras.metrics.Mean(name="g_loss")

sample_path = glob("/content/ILSVRC2013_DET_train/*.tar")
file_number = 0
for tar in sample_path:

  file_number += 1

  result_file_with_number = file_path + "file_sample_" + str(file_number)
  file_name = "/content/extracted_tar"

  if not os.path.exists(file_name):
    os.makedirs(file_name)

  if not os.path.exists(result_file_with_number):
    os.makedirs(result_file_with_number)
  else:
    continue
  
  my_tar = tarfile.open(tar)
  my_tar.extractall(file_name)
  my_tar.close()

  img_p_1 = glob(file_name + "/*/*.JPEG")
  img_p_2 = glob(file_name + "/*.JPEG")

  img_path = img_p_1 + img_p_2

  bw_image_remove(img_path)


  img_p_1 = glob(file_name + "/*/*.JPEG")
  img_p_2 = glob(file_name + "/*.JPEG")

  img_path = img_p_1 + img_p_2
 

  for epoch in range(epochs):
      for batch in tqdm(range(0, len(img_path), batch_size)):

          if len(img_path) - batch_size < batch:
              continue
          
          gray_img, color_img = lab_np(batch_size = batch_size, batch= batch, img_path = img_path)

          cg_loss, d_loss = train_step(real_imgs = color_img, gray_imgs= gray_img)

          d_losses.update_state(d_loss)
          g_losses.update_state(cg_loss)


      if epoch % show_every == 0:
          print(f"epoch : {epoch}/{epochs}  d_loss : {d_losses.result():0.5f}  g_loss: {g_losses.result():0.5f}")
          plot_result(epoch=epoch, img_path = img_path, file_number = file_number)
          d_losses.reset_states()
          g_losses.reset_states()

          check_point_manager.save()

  shutil.rmtree(file_name, ignore_errors = True)

100%|██████████| 61/61 [00:30<00:00,  1.97it/s]


epoch : 0/200  d_loss : 0.30633  g_loss: 0.04987


100%|██████████| 61/61 [00:17<00:00,  3.50it/s]


epoch : 10/200  d_loss : 0.26373  g_loss: 0.05195


100%|██████████| 61/61 [00:17<00:00,  3.50it/s]


...

100%|██████████| 18/18 [00:04<00:00,  3.63it/s]


epoch : 190/200  d_loss : 0.02915  g_loss: 0.01411


100%|██████████| 112/112 [00:32<00:00,  3.47it/s]


epoch : 0/200  d_loss : 0.68615  g_loss: 0.02092


 62%|██████▎   | 70/112 [00:20<00:12,  3.48it/s]